In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,019 kB]
Hit:13 http://ppa.launch

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-14 21:34:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.04MB/s    in 0.2s    

2023-03-14 21:34:52 (5.04 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"

spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

In [9]:
# Get the number of rows in the DataFrame.
user_data_df.count()


705889

# Transform the Data

## Create the "review_id_table".

In [10]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

review_id_table = user_data_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date('review_date')])
review_id_table = review_id_table.withColumnRenamed("to_date(review_date)","review_date")
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RY01SAV7HZ8QO|     179886|B00NTI0CQ2|     667358431| 2015-08-31|
|R1XX8SDGJ4MZ4L|   37293769|B00BUCLVZU|     621695622| 2015-08-31|
|R149Q3B5L33NN5|    8332121|B00RBX9D5W|     143071132| 2015-08-31|
|R2ZVD69Z6KPJ4O|   47054962|B00UJ3IULO|     567816707| 2015-08-31|
|R1DIKG2G33ZLNP|   23413911|B0037UCTXG|     909557698| 2015-08-31|
|R3L6FGKAW0EYFI|    4417771|B004N866SU|     414565179| 2015-08-31|
| RAO0QZH5VC6VI|   47900707|B00JE6AOJS|     798450889| 2015-08-31|
|R25IK0UAHWNB22|   34112894|B00COL0B7A|     777554234| 2015-08-31|
|R2A9IHKZMTMAL1|   20691979|B00QHLSKOE|     885228855| 2015-08-31|
| R5XVKTHL6SITI|   25983343|B00UNL2MUW|     164482798| 2015-08-31|
|R2QZZOSTDDY1IE|   35816068|B00RIC9JB4|     184834831| 2015-08-31|
|R38CUDCFPSNYTD|   10628020|B00HPMCO6O|     444378461| 2015-08

## Create the "products" Table

In [12]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = user_data_df.select(['product_id', 'product_title'])
products_df = products_df.dropDuplicates(['product_id'])
products_df = products_df.dropDuplicates(['product_title'])

products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0055B6FG2|"""the Roost"" Pl...|
|B0080CXSQ8|"Abstract" 0070, ...|
|B005587H3U|"Coffee" with Act...|
|B000ZBUHZM|"DYNAMIC" earphon...|
|B00013BN34|"Jwin Jvtv1010B 5...|
|B00GE7KXV4|"Let's Pink !"Gir...|
|B00BZ4Z5EQ|"Monkeys" 0021, H...|
|B00BZ50B5S|"Owls" 0007, High...|
|B00LFWTEBI|"T"-Black and Whi...|
|B00BZ91V9O|"Vibrant" 10016, ...|
|B00E6BWH2S|#1 Quad Core Smar...|
|B0007WBRQM|$100 A2ZCDS Gift ...|
|B002TEKV7W|&nbsp Canon EF-S ...|
|B00AOGSG74|&nbsp Nikon Coolp...|
|B00B6ZM8G2|&nbsp Sony Alpha ...|
|B00EQ27M6S|( Native Resoluti...|
|B000SOCNHQ|() Premier Acoust...|
|B008DX2TPK|(10) Triple Alpha...|
|B003MFHWGA|(13.8 Inch, 35.1 ...|
|B01042PUUI|(14 Pack) QKOO Mi...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [26]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = user_data_df.groupby("customer_id").count()
customers_df = customers_df.withColumnRenamed("count","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10142992|             1|
|   16457323|             6|
|   11935383|             1|
|   46277736|             1|
|   13671072|             1|
|   21453814|             1|
|   17684885|             1|
|   20415768|             1|
|   15212710|             1|
|    5220924|             1|
|   46253451|             6|
|     971908|             1|
|   32829933|             1|
|   51221518|             1|
|   12002637|             2|
|   16105308|             1|
|     135867|             1|
|   47425808|             1|
|   43138273|             1|
|   16411995|             1|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [13]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = user_data_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'] )

vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RY01SAV7HZ8QO|          4|            0|          0|   N|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|
|R149Q3B5L33NN5|          5|            0|          0|   N|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|
|R3L6FGKAW0EYFI|          1|            1|          1|   N|
| RAO0QZH5VC6VI|          1|            0|          0|   N|
|R25IK0UAHWNB22|          3|            0|          0|   N|
|R2A9IHKZMTMAL1|          5|            1|          2|   N|
| R5XVKTHL6SITI|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



# Load

In [22]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb1.cigtzod05pek.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", "password": "381634Cm!", "driver":"org.postgresql.Driver"}

In [23]:
# Write review_id_df to table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [24]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [27]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)


In [28]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)